# Basic membrane representation

In [1]:
import numpy as np
from numba import cuda
from collections import defaultdict

In [15]:
EVOLVE = 0
SENDIN = 1
SENDOUT = 2
DISSOL = 3
DIVIDE = 4

In [89]:
class PMembrane(object):
    def __init__(self, O=[], multiset=''):
        self.O = O
        self.H = [1]                                  # Label of membranes (skin)
        self.mu = [-1]                                # Tree with indices (membranes will be 0-indexed)
                                                      # mu: membrane -> parent
        self.omega = [multiset]                       # Multiset of objects placed in the regions of mu

#         self.rules_evolution = defaultdict(list)      # (membrane, object) -> object
#         self.rules_sendin = defaultdict(list)         # (membrane, object) -> object
#         self.rules_sendout = defaultdict(list)        # (membrane, object) -> object
#         self.rules_dissolution = defaultdict(list)    # (membrane, object) -> object
#         self.rules_division = defaultdict(list)       # (membrane, object) -> (object, object) (simplify to object)
        
        self.rules = [defaultdict(list) for i in range(5)]
        # [evolution, sendin, sendout, dissolution, division]

    def add_membrane(self, parent, label, multiset=''):
        self.H.append(label)
        self.mu.append(parent)
        self.omega.append(multiset)
        
    def add_rule(self, membrane_label, obj, ruletype, new_obj):
        self.rules[ruletype][(membrane_label, obj)] = new_obj
        
    def level2_membranes(self):
        return [ndx for ndx, parent in enumerate(self.mu) if parent == 0]
    
    def level3_membranes(self):
        return [ndx+1 for ndx, parent in enumerate(self.mu[1:]) if self.mu[parent] == 0]
    
    def level2_submembranes(self, membrane):
        if self.mu[membrane] == 0:
            
            # Get list of submembranes
            submembranes = [ndx for ndx, parent in enumerate(self.mu) if parent == membrane]
            matrix = np.zeros((len(submembranes), len(self.O)),dtype=np.int16)
            membranes = []
            
            for ndx, sm in enumerate(submembranes):
                membranes.append(sm)
                for obj in self.omega[sm]:
                    matrix[ndx, self.O.index(obj)] += 1

            return matrix, membranes
        
    def level2_multiset(self, membrane):
        if self.mu[membrane] == 0:
            matrix = np.zeros(len(self.O), dtype=np.int16)
            for obj in self.omega[membrane]:
                matrix[self.O.index(obj)] += 1
            
            return matrix
        
    def ruleset(self):
        # array of tuples: (object, membrane, ruletype, output object)
        # for simplicity, division rules will only output 2 membranes with the same evolved object
        ruleset = []
        for i in range(5):
            for k, v in self.rules[i].items():
                ruleset.append((
                    self.O.index(k[1]),
                    k[0],
                    i,
                    self.O.index(v)
                ))
        ruleset.sort()
        objects = [r[0] for r in ruleset]
        indices = [objects.index(i) if i in objects else -1 for i in range(len(self.O))]
        return ruleset, indices

In [90]:
# Test membrane

test = PMembrane(O=['a', 'b', 'c'], multiset="a")
test.add_membrane(0, 2, multiset="abc")
test.add_membrane(0, 2, multiset="aab")
test.add_membrane(0, 2, multiset="abcc")
test.add_membrane(1, 3, multiset="a")
test.add_membrane(1, 3, multiset="bb")
test.add_membrane(1, 3, multiset="c")
test.add_membrane(2, 3, multiset="ac")
test.add_membrane(2, 3, multiset="bbc")
test.add_membrane(2, 3, multiset="ccc")
test.add_membrane(3, 3, multiset="aa")
test.add_membrane(3, 3, multiset="bc")
test.add_membrane(3, 3, multiset="b")

test.add_rule(3, 'a', EVOLVE, 'b')
test.add_rule(3, 'b', EVOLVE, 'c')
test.add_rule(2, 'a', EVOLVE, 'b')
test.add_rule(2, 'b', EVOLVE, 'c')

In [91]:
test.level2_membranes()

[1, 2, 3]

In [98]:
test.level2_submembranes(1)

(array([[1, 0, 0],
        [0, 2, 0],
        [0, 0, 1]], dtype=int16), [4, 5, 6])

In [95]:
test.level3_membranes()

[4, 5, 6, 7, 8, 9, 10, 11, 12]

In [96]:
test.level2_multiset(2)

array([2, 1, 0], dtype=int16)

In [97]:
# (object, membrane, ruletype, output object)
test.ruleset()

([(0, 2, 0, 1), (0, 3, 0, 1), (1, 2, 0, 2), (1, 3, 0, 2)], [0, 2, -1])

# Selection / Evolution Kernel

In [ ]:
@cuda.jit
def selection_kernel(M_3, M_2, R, R_ndx, out):
    obj = cuda.threadIdx.x
    mem_3 = cuda.threadIdx.y
    mem_2 = cuda.blockIdx.x
    
    rulecount = len(R)
    
    # Select level 3 rules first
    if R_ndx[mem] >= 0 and M_3[mem_2][mem_3][obj] > 0:
        for i in range(R_ndx[mem], rulecount):
            if (R[i][0] != obj):
                break
            if (R[i][1] == 3):
                out[i] = 1
            # select level 2's as well, if they're in the thing
            if (R[i][1] == 2 and M_2[mem_2][obj] > 0):
                out[i] = 1